# Nachbildung Mathematischer Operationen durch ML-Modelle

## 1. Erstellung der Datensätze

In [ ]:
import numpy as np
import pandas as pd

N = 100000

x1 = np.random.uniform(-10, 10, N)
x2 = np.random.uniform(-10, 10, N)

y = x1 + x2   # oder x1 * x2, x1 - x2

data = pd.DataFrame({
    "x1": x1,
    "x2": x2,
    "y": y
})

data.to_csv("addition_dataset.csv", index=False)


## 2. SVM 

### 2.1 Addition

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np

# Load the dataset
data = pd.read_csv("addition_dataset.csv")

X = data[["x1", "x2"]].values
y = data["y"].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


svr = SVR(kernel="linear", C=10, epsilon=0.01)
svr.fit(X_train, y_train)

# Make predictions on Test set
y_pred = svr.predict(X_test)


# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
relative_error = np.mean(np.abs(y_test - y_pred) / (np.abs(y_test) + 1e-8))

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Relative Error: {relative_error}")

print("C:", svr.C)
print("Epsilon:", svr.epsilon)
print("Kernel:", svr.kernel)
print("Gamma:", svr.gamma if hasattr(svr, "gamma") else None)


Mean Squared Error: 1.732199741685354e-05
Mean Absolute Error: 0.0034251641630859268
Mean Relative Error: 0.0013018782870338798
C: 10
Epsilon: 0.01
Kernel: linear
Gamma: scale


### 2.2 Multiplikation

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np

# Load the dataset
data = pd.read_csv("multiplication_dataset.csv")

data = data.sample(n=20000, random_state=42)


X = data[["x1", "x2"]].values
y = data["y"].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


svr = SVR(kernel="rbf", C=10, gamma="scale", epsilon=0.01)
svr.fit(X_train, y_train)

# Make predictions on Test set
y_pred = svr.predict(X_test)


# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
relative_error = np.mean(np.abs(y_test - y_pred) / (np.abs(y_test) + 1e-8))

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Relative Error: {relative_error}")

print("C:", svr.C)
print("Epsilon:", svr.epsilon)
print("Kernel:", svr.kernel)
print("Gamma:", svr.gamma if hasattr(svr, "gamma") else None)

Mean Squared Error: 0.030964836792467337
Mean Absolute Error: 0.04658233828763949
Mean Relative Error: 0.0021908643269368507
C: 10
Epsilon: 0.01
Kernel: rbf
Gamma: scale
